In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, when, first, avg
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql import DataFrame

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# Carregar tabelas
unemployment_df = spark.table("database.gender")
wage_df = spark.table("database.wage")

# Lista de países e anos
paises = ["%Germany%", "%Denmark%", "%Switzerland%", "%Poland%", "%Czech%", "%Slovenia%", "%Hungary%", "%Austria%"]
anos = ["2010", "2011", "2012", "2013", "2014", "2015"]

# Filtrando os DataFrames

def filter_by_multiple_conditions(df: DataFrame, paises: list, anos: list) -> DataFrame:
    conditions = reduce(lambda a, b: a | b, (col("nome_pais").like(p) for p in paises))
    df = df.filter(conditions)
    df = df.filter(col("ano").isin(anos))
    return df

unemployment_df = filter_by_multiple_conditions(unemployment_df, paises, anos)
wage_df = filter_by_multiple_conditions(wage_df, paises, anos)

unemployment_df = unemployment_df.drop("Country_code", "Country_name", "Indicator_name", "Indicator_code", "cod_indicador", "cod_pais")

wage_df = wage_df.drop("cod_indicador", "cod_pais")

wage_df = wage_df.withColumnRenamed('Disaggregation', 'desagregacao')

wage_df = wage_df.filter((col('desagregacao') == 'female') | (col('desagregacao') == 'male'))

# Filtrar pelos indicadores de emprego
unemployment_df = unemployment_df.filter(F.col("nome_indicador").contains("Unemployment") & ~F.col("nome_indicador").contains("with"))

# Adicionar a coluna "genero" com base no nome do indicador
unemployment_df = unemployment_df.withColumn("desagregacao",
    F.when(F.col("nome_indicador").like("% female%"), "female")
    .when(F.col("nome_indicador").like("% male%"), "male")
)

unemployment_df = unemployment_df.filter(F.col("desagregacao").isNotNull())


wage_df = wage_df.filter(F.col("desagregacao").isNotNull())

# Calcular a média dos valores por ano, gênero e indicador
wage_df_media = wage_df.groupBy("nome_pais", "ano", "desagregacao", "nome_indicador") \
    .agg(F.avg("valor").alias("valor"))

unemployment_df_media = unemployment_df.groupBy("nome_pais", "ano", "desagregacao", "nome_indicador") \
    .agg(F.avg("valor").alias("valor"))

# Selecionar apenas uma linha por ano e gênero
window = Window.partitionBy("nome_pais", "ano", "desagregacao").orderBy(F.col("valor"))

wage_df_final = wage_df_media.withColumn("row_number", F.row_number().over(window)) \
    .filter(F.col("row_number") == 1).drop("row_number")

unemployment_df_final = unemployment_df_media.withColumn("row_number", F.row_number().over(window)) \
    .filter(F.col("row_number") == 1).drop("row_number")

# Definindo a ordem das colunas em 
columns_order = ["nome_indicador", "nome_pais", "ano", "valor", "desagregacao"]

# Reorganizando as colunas no DataFrame 
wage_df_final = wage_df_final.select(columns_order)
unemployment_df_final = unemployment_df_final.select(columns_order)

In [3]:
wage_df_final = wage_df_final.withColumn("valor", col("valor").cast("float"))
unemployment_df_final = unemployment_df_final.withColumn("valor", col("valor").cast("float"))

In [4]:
df_concatenado = wage_df_final.union(unemployment_df_final)
df_concatenado = df_concatenado.withColumn("ano", col("ano").cast("int"))

In [5]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

df_concatenado.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaWageUnemploymentGold")

In [6]:
spark.sql(
    """
    SELECT * FROM database.WageUnemploymentGold
    """
).show(500)

+-----------+--------------------+----+--------+------------+
|  nome_pais|      nome_indicador| ano|   valor|desagregacao|
+-----------+--------------------+----+--------+------------+
|    Austria|Unemployment, fem...|2010|   4.632|      female|
|    Austria|Unemployment, mal...|2010|   4.982|        male|
|    Austria|Unemployment, fem...|2011|   4.576|      female|
|    Austria|Unemployment, mal...|2011|   4.546|        male|
|    Austria|Unemployment, fem...|2012|   4.795|      female|
|    Austria|Unemployment, mal...|2012|   4.936|        male|
|    Austria|Unemployment, fem...|2013|   5.297|      female|
|    Austria|Unemployment, mal...|2013|   5.378|        male|
|    Austria|Unemployment, fem...|2014|   5.381|      female|
|    Austria|Unemployment, mal...|2014|   5.831|        male|
|    Austria|Unemployment, fem...|2015|   5.306|      female|
|    Austria|Unemployment, mal...|2015|   6.085|        male|
|    Czechia|Unemployment, fem...|2010|   8.459|      female|
|    Cze

In [7]:
spark.stop()